In [ ]:
import pandas as pd
import sqlite3
import os

# connecting to existing database
base_dir = os.path.expanduser("~/iCloud Drive/Desktop/storage-company-data")
if not os.path.exists(base_dir):
    base_dir = os.path.expanduser("~/Desktop/storage-company-data")
db_path = os.path.join(base_dir, "Data", "storage_company.db")

conn = sqlite3.connect(db_path)
customers = pd.read_sql_query("SELECT * FROM customers;", conn)


In [3]:
print(customers.columns.tolist())


['customer_id', 'first_name', 'last_name', 'email', 'phone', 'street_address', 'city', 'postal_code', 'country', 'sign_up_date', 'lease_start', 'lease_end', 'unit_id', 'unit_size', 'monthly_fee', 'payment_status', 'last_payment_date', 'notes']


In [4]:
# ---- NUMERIC COLUMNS ----
for col in ["monthly_fee", "last_payment_amount"]:
    if col in customers.columns:
        customers[col] = pd.to_numeric(customers[col], errors="coerce")


In [5]:
if "monthly_fee" in customers.columns:
    customers.rename(columns={"monthly_fee": "unit_monthly_fee"}, inplace=True)


In [6]:
customers.sample(5)


,customer_id,first_name,last_name,email,phone,street_address,city,postal_code,country,sign_up_date,lease_start,lease_end,unit_id,unit_size,unit_monthly_fee,payment_status,last_payment_date,notes
2037,C101439,Elijah,Anderson,eanderson@yahoo.com,+31 6 68277654,Baker St 160,Leiden,8716 AB,Netherlands,2025-04-19,NaT,2023-11-12,404a,3 sqm,199.36,suspended,NaT,Prefers email
619,C102122,Lucas,Johnson,ljohnson@inboxstorage.eu,06-16928614,Main St 53,Amsterdam,9223,Netherlands,NaT,2020-07-31,NaT,u1180,8 sqm,434.88,paid,2022-09-17,Key lost
847,C101917,Emma,Jackson,emmaja@outlook.com,981186761 ext 971,Hoofdstraat 20,Leiden,5634 AA,Netherlands,NaT,NaT,NaT,u1052,—,73.18,Overdue,NaT,Moved out
534,C100963,Benjamin,Garcia,None,156820727,Princenhagelaan 147,Haarlemmermeer,5556 AA,Netherlands,NaT,NaT,NaT,u1198,5m2,420.39,PAID,NaT,警告: unpaid
905,C101085,James,Lopez,jameslo@inboxstorage.eu,+31 6 74111213,Dorpsweg 5,Leiden,3074 AA,Netherlands,2017-11-03,NaT,NaT,u1374,6 sqm,157.15,Overdue,NaT,Prefers email


In [ ]:
import re
import numpy as np
from datetime import datetime

# ---- STANDARDIZING COUNTRY ----
customers["country"] = (
    customers["country"]
    .str.strip()
    .str.replace("Nederland", "Netherlands", regex=False)
    .str.replace("Ned.", "Netherlands", regex=False)
    .str.replace("NL", "Netherlands", regex=False)
)

# ---- STANDARDIZING PAYMENT STATUS ----
customers["payment_status"] = (
    customers["payment_status"]
    .astype(str)
    .str.strip()
    .str.lower()
    .replace({
        "paid": "paid",
        "overdue": "overdue",
        "suspended": "suspended"
    })
)

# ---- CLEANING PHONE ----
customers["phone"] = (
    customers["phone"]
    .astype(str)
    .str.replace(r"[^\d+]", "", regex=True)  # remove letters and symbols
    .str.replace(r"^31", "+31", regex=True)  # ensure country code prefix
)

# ---- CLEANING UNIT SIZE ----
def extract_sqm(value):
    if pd.isna(value): 
        return np.nan
    match = re.search(r"(\d+(\.\d+)?)", str(value))
    return float(match.group(1)) if match else np.nan

customers["unit_size"] = customers["unit_size"].apply(extract_sqm)

# ---- FIXING DATES ----
for col in ["sign_up_date", "lease_start", "lease_end", "last_payment_date"]:
    customers[col] = pd.to_datetime(customers[col], errors="coerce", dayfirst=True)

# ---- ENSURING NUMERIC MONTHLY FEE ----
customers["unit_monthly_fee"] = pd.to_numeric(customers["unit_monthly_fee"], errors="coerce")

print("Standardization done.")


Standardization done.


In [9]:
customers.sample(5)


,customer_id,first_name,last_name,email,phone,street_address,city,postal_code,country,sign_up_date,lease_start,lease_end,unit_id,unit_size,unit_monthly_fee,payment_status,last_payment_date,notes
1478,C101464,Ava,Gonzalez,ava.gonzalez@hotmail.com,+31655457465,Dorpsweg 108,Utrecht,7539 B,Netherlands,2020-11-29,NaT,NaT,298a,4.0,506.88,overdue,NaT,Customer asked for discount
1306,C101489,Mia,Garcia,mgarcia@hotmail.com,+31620625528,Oudeweg 109,Leiden,1275,Netherlands,2020-02-22,NaT,2021-12-09,unit 479,15.0,74.71,paid,NaT,Prefers email
516,C100613,Sophia,Davis,sophia.davis@yahoo.com,0639871484,Stationsplein 181,Leiden,4225 AA,Netherlands,NaT,NaT,2027-02-15,207a,10.0,458.46,paid,NaT,警告: unpaid
1197,C100573,Ava,Williams,awilliams@yahoo.com,+31623481094,Dorpsweg 60,Rotterdam,5104 AA,Netherlands,NaT,NaT,NaT,u1040,NaN,548.00,suspended,NaT,None
1850,C101346,Harper,Taylor,htaylor@outlook.com,947,Main St 187,Eindhoven,9544 AC,Netherlands,NaT,2018-05-23,NaT,unit 381,15.0,239.49,overdue,NaT,警告: unpaid
